In [ ]:
## Using UCI's Individual Household Electric Power Consumption Data

https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption
    
## Create Flask App:

* Against sqlite database utilize SQLAlchemy on one object PowerConsumption
* 


 Action | URL | Method
 - | - | -
 Destructive refresh | http://127.0.0.1:5000/v1.0/ihepc/refresh | DELETE 
 Retrieves based upon supplied attribute filters | http://127.0.0.1:5000/v1.0/ihepc/ query params | GET 
 Updates existing records defined by "id." Need to provide JSON with the new values | http://127.0.0.1:5000/v1.0/ihepc  id:int| PUT 
 Add new record. Need to provide JSON of record | http://127.0.0.1:5000/v1.0/ihepc | POST 
 Deletes specific record | http://127.0.0.1:5000/v1.0/ihepc/ id:int | DELETE 

#### What refresh does

* Pulls zip from url
* Places in inbound directory giving it a unique name
* For processing:
 * Moves file into processing directory
 * Truncates power_consumption table
 * Uses ZipFile for reading lines of file Bulk inserts using SQLAlchemy
 * Then, files is moved into archived directory
 * Download takes ~1min
 * Bulk insert tables ~1min

#### Query Parameters for Getting/Retrieving data follow certain format

< attribute > = < operator > : < value >
    
##### Valid Operators:
| operator | definition |
| - | - |
| gte | greater than or equal |
| lte | less than or equal |
| gt | greater than |
| lt | less than |
| eq | equal |

##### Limitiations:

- Each filter is 'and'ed together
- Can't use attribute more than once
- Doesn't support 'from' 'to' which would be beneficial for dates

#### Sqlite SQL Alchemy Power Consumption Object:

`class PowerConsumption(Base):
    __tablename__ = "power_consumption"
    __table_args__ = {'sqlite_autoincrement': True}
    id = Column(Integer, primary_key=True)
    date = Column(Date, nullable=False)
    date_time = Column(DateTime, nullable=False)
    global_active_power = Column(Float)
    global_reactive_power = Column(Float)
    voltage = Column(Float)
    global_intensity = Column(Float)
    sub_metering_1 = Column(Float)
    sub_metering_2 = Column(Float)
    sub_metering_3 = Column(Float)
    line_no = Column(Integer)
    UniqueConstraint('date_time', name='power_date_time_unq')
    Index('power_date_time_idx', 'date_time')
    Index('power_date_idx', 'date')
`



In [8]:
import requests
import json

url_refresh = "http://127.0.0.1:5000/v1.0/ihepc/refresh"
url_ihepc = "http://127.0.0.1:5000/v1.0/ihepc"

In [9]:
!date
res = requests.request('DELETE', url_refresh)
!date
res

Thu Feb 14 22:36:11 PST 2019
Thu Feb 14 22:38:25 PST 2019


<Response [200]>

`sqlite> select count(*) from power_consumption ;
2075259
`

### Example of Query

In [10]:
# Querying
data_filters = "sub_metering_3=gte:16.0&global_intensity=gte:39.4&date=eq:2009-11-24"

!date
res = requests.request('GET', url_ihepc, params=data_filters)
!date
res 

Thu Feb 14 22:38:25 PST 2019
Thu Feb 14 22:38:26 PST 2019


<Response [200]>

In [11]:
res.json()

[[{'date': '2009-11-24',
   'date_time': '2009-11-24T19:34:00+00:00',
   'global_active_power': 9.718,
   'global_intensity': 41.8,
   'global_reactive_power': 0.256,
   'id': 22399292,
   'line_no': 1546692.0,
   'sub_metering_1': 37.0,
   'sub_metering_2': 70.0,
   'sub_metering_3': 17.0,
   'voltage': 231.89},
  {'date': '2009-11-24',
   'date_time': '2009-11-24T19:35:00+00:00',
   'global_active_power': 9.714,
   'global_intensity': 42.0,
   'global_reactive_power': 0.358,
   'id': 22399293,
   'line_no': 1546693.0,
   'sub_metering_1': 36.0,
   'sub_metering_2': 69.0,
   'sub_metering_3': 18.0,
   'voltage': 231.41},
  {'date': '2009-11-24',
   'date_time': '2009-11-24T19:36:00+00:00',
   'global_active_power': 9.726,
   'global_intensity': 42.0,
   'global_reactive_power': 0.37,
   'id': 22399294,
   'line_no': 1546694.0,
   'sub_metering_1': 36.0,
   'sub_metering_2': 70.0,
   'sub_metering_3': 17.0,
   'voltage': 231.63},
  {'date': '2009-11-24',
   'date_time': '2009-11-24T19:

### Example of Delete
* sqlite query shows before and after

In [12]:
!date
res = requests.request('DELETE', "%s/%d" % (url_ihepc,20324033))
!date
res  

Thu Feb 14 22:38:26 PST 2019
Thu Feb 14 22:38:26 PST 2019


<Response [200]>

`sqlite> select * from power_consumption where id = 20324033 ;
20324033|2009-11-24|2009-11-24 19:34:00.000000|9.718|0.256|231.89|41.8|37.0|70.0|17.0|1546692
sqlite> select * from power_consumption where id = 20324033 ;
sqlite>`

#### Example of Update
* sqlite query shows before and after

In [6]:
update = {
    'global_active_power': 55.708,
    'global_intensity': 55.0,
    'global_reactive_power': 0.555,
    'line_no': 1546697.0,
    'sub_metering_1': 55.0,
    'sub_metering_2': 55.0,
    'sub_metering_3': 55.0,
    'voltage': 555.55}

!date
res = requests.request('PUT', "%s/%d" % (url_ihepc,20324037), json=json.dumps(update))
!date
res  

Thu Feb 14 21:48:01 PST 2019
Thu Feb 14 21:48:02 PST 2019


<Response [200]>

`sqlite> select * from power_consumption where id = 20324037;
20324037|2009-11-24|2009-11-24 19:38:00.000000|9.726|0.364|231.61|42.0|36.0|70.0|17.0|1546696
sqlite> select * from power_consumption where id = 20324037;
20324037|2009-11-24|2009-11-24 19:38:00.000000|55.708|0.555|555.55|55.0|55.0|55.0|55.0|1546696
`

In [ ]:
new_data_point = {
    'date': '2020-11-24',
    'date_time': '2020-11-24T19:39:00+00:00',
    'global_active_power': 9.708,
    'global_intensity': 42.0,
    'global_reactive_power': 0.372,
    'line_no': 1546697.0,
    'sub_metering_1': 37.0,
    'sub_metering_2': 69.0,
    'sub_metering_3': 17.0,
    'voltage': 231.37}

res = requests.request('POST', url_ihepc, json=json.dumps(new_data_point))

In [ ]:
### Improvements:

* Lacks security (api key, oauth...)
* CRUD works on single items: would be nice to handle lists
* Query/Get lacks any paging functionality
* Error handling; argument validation
* Bug in CREATE